In [1]:
import numpy as np
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Pi") \
        .getOrCreate()

# You can examine the full config
spark.sparkContext.getConf().getAll()

ss=SparkSession.builder.master("local").getOrCreate()

sc=spark.sparkContext.getConf()
print(ss.version)

import sys
from random import random
from operator import add

def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

partitions = 2
n = 10000 * partitions
count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
print("Pi is roughly %f" % (4.0 * count / n))

spark.stop()

21/09/03 14:02:48 WARN Utils: Your hostname, bruce-NUC8i7BEH resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlp0s20f3)
21/09/03 14:02:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/03 14:02:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


3.1.2


Pi is roughly 3.140800


In [2]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("Word2VecExample")\
    .getOrCreate()

# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))
# $example off$

spark.stop()

21/09/03 14:03:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/09/03 14:03:23 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Text: [Hi, I, heard, about, Spark] => 
Vector: [0.07286177035421133,0.03824751451611519,0.014087121188640596]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.00646721278982503,-0.006787749805620738,-0.017842545041016167]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.009297877550125122,0.07595592737197876,-0.06726421769708395]



In [4]:
from pyspark.ml.feature import RFormula

from pyspark.ml.regression import LinearRegression

from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("RFormulaExample")\
    .getOrCreate()

# $example on$
dataset = spark.createDataFrame(
    [(7, "US", 18, 1.0),
     (8, "CA", 12, 0.0),
     (9, "NZ", 15, 0.0)],
    ["id", "country", "hour", "clicked"])

formula = RFormula(
    formula="clicked ~ country + hour",
    featuresCol="features",
    labelCol="label")

output = formula.fit(dataset).transform(dataset)
output.select("features", "label").show()
# $example off$

lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(output)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)


spark.stop()

+--------------+-----+
|      features|label|
+--------------+-----+
|[0.0,0.0,18.0]|  1.0|
|[1.0,0.0,12.0]|  0.0|
|[0.0,1.0,15.0]|  0.0|
+--------------+-----+

Coefficients: [0.0,0.0,0.06093901400932439]
Intercept: -0.5807518768065326
numIterations: 10
objectiveHistory: [0.5, 0.48867004169790873, 0.4497217673145075, 0.4443566163778985, 0.44361756323553725, 0.4435157581691582, 0.44350173445293756, 0.44349980267661604, 0.44349953657313135, 0.4434994999171975, 0.44349949486781826]
+--------------------+
|           residuals|
+--------------------+
|  0.4838496246386935|
|-0.15051629130536004|
|-0.33333333333333315|
+--------------------+

RMSE: 0.350179
r2: 0.448185
